# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [2]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1614565773.0
Print triggered 1614565773.0
Main triggered 1614565773.5
Main triggered 1614565774.0
Print triggered 1614565774.0


# Controller Base Class (eFMU)

In [4]:
# Test controller
testcontroller = testcontroller1() #this controller multiplies the inputs
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = random.randint(1,50)
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 7, 'b': 4}
Output {'c': 28}
Output {'c': 28}


# Controller Stack Class (single-thread/multi-thread)

In [5]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [6]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
for i in range(6):
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

Exception ignored in: <function _ConnectionBase.__del__ at 0x7fe6d3d9eaf0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/connection.py", line 137, in __del__
    self._close()
KeyboardInterrupt: 


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:41.980774912,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:44.261884928,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:46.543813120,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:48.821138944,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-28 18:29:51.100631040,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:41.980774912,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:44.261884928,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:46.543813120,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:48.821138944,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-28 18:29:51.100631040,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:41.980774912,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:44.261884928,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:46.543813120,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:48.821138944,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-02-28 18:29:51.100631040,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:41.980774912,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:44.261884928,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:46.543813120,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:48.821138944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-28 18:29:51.100631040,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-28 18:29:32.457709056,NaN,NaN,NaN,Initialize
2021-02-28 18:29:39.692802048,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:41.980774912,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:44.261884928,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:46.543813120,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:48.821138944,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-28 18:29:51.100631040,30.0,4.0,120.0,testcontroller1 did a computation!


13.827431201934814


## Multi Thread

In [4]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)


# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
while time.time() - t < 50:
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

Controller timeout forecast2


/Users/deoxi/FMLC/FMLC/stackedclasses.py:285: Warning: Controller forecast2 timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


Controller timeout control1


/Users/deoxi/FMLC/FMLC/stackedclasses.py:285: Warning: Controller control1 timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


,a,b,c,Logging
2021-03-02 00:32:15.553690112,NaN,NaN,NaN,Initialize
2021-03-02 00:33:01.459888896,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:02.714194944,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:03.964910080,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:05.214654976,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:06.459323904,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:07.717541888,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:08.971579904,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:10.218350080,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:33:11.460958976,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-02 00:32:15.553690112,NaN,NaN,NaN,Initialize
2021-03-02 00:33:01.459888896,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:02.714194944,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:03.964910080,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:05.214654976,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:06.459323904,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:07.717541888,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:08.971579904,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:10.218350080,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:33:11.460958976,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-02 00:32:15.553690112,NaN,NaN,NaN,Initialize
2021-03-02 00:33:01.459888896,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:02.714194944,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:03.964910080,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:05.214654976,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:06.459323904,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:07.717541888,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:08.971579904,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:10.218350080,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:33:11.460958976,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-02 00:32:15.553690112,NaN,NaN,NaN,Initialize
2021-03-02 00:33:01.459888896,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:02.486404096,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:03.507436032,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:04.520967936,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:05.540520960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:06.560819968,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:07.576980992,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:08.607980032,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:33:09.642639104,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-02 00:32:15.553690112,NaN,NaN,NaN,Initialize
2021-03-02 00:33:01.459888896,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:02.460593920,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:03.462435072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:04.464915968,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:05.465309952,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:06.465359104,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:07.468195072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:08.470024960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:33:09.470212096,30.0,4.0,120.0,testcontroller1 did a computation!


52.174509048461914


In [3]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)
print("done initializing")

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
ctrl_stack.run_query_control_for(10)
print(time.time() - t)

done initializing
10.051069259643555


In [4]:
for df in ctrl_stack.log_to_df().values():
    display(df)

,a,b,c,Logging
2021-03-02 00:56:10.838565120,NaN,NaN,NaN,Initialize
2021-03-02 00:56:21.362454016,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:56:22.649786112,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:56:23.929896960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:56:25.221821952,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:56:26.508569088,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:56:27.784218880,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:56:29.140974080,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-02 00:56:30.431347968,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-02 00:56:10.838565120,NaN,NaN,NaN,Initialize
2021-03-02 00:56:21.362454016,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:56:22.649786112,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:56:23.929896960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:56:25.221821952,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:56:26.508569088,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:56:27.784218880,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:56:29.140974080,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-02 00:56:30.431347968,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-02 00:56:10.838565120,NaN,NaN,NaN,Initialize
2021-03-02 00:56:21.362454016,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:56:22.649786112,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:56:23.929896960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:56:25.221821952,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:56:26.508569088,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:56:27.784218880,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:56:29.140974080,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-02 00:56:30.431347968,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-02 00:56:10.838565120,NaN,NaN,NaN,Initialize
2021-03-02 00:56:21.362454016,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:22.440283904,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:23.505140992,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:24.575049984,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:25.598156032,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:26.669200896,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:27.733213952,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:28.754683904,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-02 00:56:29.788271104,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-02 00:56:10.838565120,NaN,NaN,NaN,Initialize
2021-03-02 00:56:21.362454016,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:22.387555072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:23.402149120,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:24.417187072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:25.440642048,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:26.456268032,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:27.466333952,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:28.495788032,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-02 00:56:29.517437952,30.0,4.0,120.0,testcontroller1 did a computation!


In [9]:
import time
import concurrent.futures
def f(s):
    time.sleep(s)
    return s
e = concurrent.futures.ThreadPoolExecutor(4)
s = range(10)
for i in e.map(f, s):
    print(i)

0
1
2
3
4
5
6
7
8
9
